In [140]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFDirectoryLoader, PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate

from langchain.tools import Tool
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.chains import retrieval_qa

from langchain.llms import Ollama
from langchain_groq import ChatGroq

from langchain_community.tools.asknews import AskNewsSearch
from langchain_community.tools.reddit_search.tool import RedditSearchRun

from dotenv import load_dotenv
import os
from typing import List, Dict, Any

In [141]:
# setting environment variables
load_dotenv()
groq_api_key = os.environ['GROQ_API_KEY']
asknews_client_id = os.environ['ASKNEWS_CLIENT_ID']
asknews_client_secret = os.environ['ASKNEWS_CLIENT_SECRET']

In [142]:
# Retrieve website content
link = input("Enter link: ")
loader = WebBaseLoader(link)

In [143]:
# Retrieve and chunk
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
final_docs = text_splitter.split_documents(documents)
title = final_docs[0].metadata['title']

In [144]:
# Store in vector database
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(final_docs, embeddings)
retriever = vectordb.as_retriever()

In [145]:
# Initialize llm
llm = ChatGroq(api_key=groq_api_key, model="gemma2-9b-it", temperature=0.4)

In [146]:
# define tools
asknews_tool = AskNewsSearch()
og_retriever_tool = create_retriever_tool(
    retriever, 
    "og_retriever",
    "Searches and returns information from the original news article. Use this to get specific facts or quotes from the source text."
)

tools = [asknews_tool, og_retriever_tool]

In [147]:
# Get neutral query/keywords
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a neutralizer assistant that converts news headline into a neutral, bias-free search query suitable for finding balanced articles on the topic. Focus only on the core subject matter and important details."),
    ("human", "Headline {headline}"),
    ("system", "Neutral Query: ")
])

chain = prompt | llm
query = chain.invoke({"headline" : title})

In [148]:
# define function to combine all outputs
def all_source_result(query: str) -> str:
    """
    Searches for information from all available sources: AskNews, Reddit, and the original article.
    Combines the results into a single context string for the LLM.
    """

    asknews_results = asknews_tool.run(query)

    og_article_docs = retriever.invoke(query)

    combined_results = (
        f"--- AskNews Results ---\n{asknews_results}\n\n"
        f"--- Original Article Context ---\n{og_article_docs}"
    )

    return combined_results

In [149]:
# create final tool
unified_search_tool = Tool(
    name="all_source_search",
    func=all_source_result,
    description="A comprehensive search tool that queries AskNews, Reddit, and the original article to find relevant information on a topic. Use this to gather all possible context before writing a final article."
)

In [150]:
# agent prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert news editor. Your task is to neutralize a biased news article. First, you will use the 'All_Source_Search' tool with a neutral query to gather a wide range of information. Then, using all the retrieved context, you will synthesize a new, unbiased, and comprehensive news article. Your final output must be a well-structured news article with all detail without bias, not a summary or a list of facts."),
    ("user", "Here is the query: {query}. Use the 'unified_search_tool' to complete the context."),
    ("placeholder", "{agent_scratchpad}")
])

In [151]:
agent = create_tool_calling_agent(llm, [unified_search_tool, asknews_tool], agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=[unified_search_tool, asknews_tool], verbose=True)

In [152]:
agent_executor.invoke({"query": query})



> Entering new AgentExecutor chain...


APIError: tool call validation failed: parameters for tool all_source_search did not match schema: errors: [missing properties: '__arg1']